In [1]:
import numpy as np
import math
from qutip import*
import matplotlib.pyplot as plt
import itertools as it
from qutip.qip.circuit import Gate
from qutip.qip.circuit import QubitCircuit, Gate
from qutip import tensor, basis
sqrt = np.sqrt
pi   = np.pi
cos  = np.cos
sin  = np.sin
exp  = np.exp

In [2]:
dissipation= [30e-6, 35e-6, 40e-6, 45e-6]
dephasing=   [1.5*30e-6, 1.5*35e-6, 1.5*45e-6, 1.5*55e-6]

gate_time_Paulis = 2.00e-8
gate_time_CZ     = 2.71e-7

In [3]:
OmegaX  = pi/gate_time_Paulis
OmegaCZ = pi/gate_time_CZ


def HPx_coeff(t, args):
    if t<2*gate_time_Paulis:
        return OmegaX/2
    else:
        return 0

    
def HPy_coeff(t, args):
    if t<2*gate_time_Paulis:
        return OmegaX/2
    else:
        return 0

    
def HPz_coeff(t, args):
    if t<2*gate_time_Paulis:
        return OmegaX/2
    else:
        return 0

    
def HCZ_coeff(t, args):
    if t<gate_time_CZ:
        return OmegaCZ
    else:
        return 0

In [4]:
t= np.linspace(0,gate_time_CZ,1000)
HX= sigmax()
HY= sigmay()
HZ= sigmaz()

PauliX_propagators=[]
PauliY_propagators=[]
PauliZ_propagators=[]
unit_propagators=[]

for i in range(len(dissipation)):
    gamma= dissipation[i]
    Gamma= dephasing[i]
    c_ops=[]
    c_ops.append(sqrt(1/gamma)*sigmam())
    c_ops.append(sqrt(1/Gamma)*sigmaz())
    
    
    H = [[HX, HPx_coeff]]
    PauliX_propagators.append(propagator(H, t, c_op_list=c_ops))
    
        
    H = [[HY, HPy_coeff]]
    PauliY_propagators.append(propagator(H, t, c_op_list=c_ops))
    
    
    H = [[HZ, HPz_coeff]]
    PauliZ_propagators.append(propagator(H, t, c_op_list=c_ops))
    
    
    H= qeye(2)
    unit_propagators.append(propagator(H, t, c_op_list=c_ops))

In [5]:
# Saving all the propagators

import pickle
with open('Data\PauliX', 'wb') as PX:
    pickle.dump(PauliX_propagators, PX)

    
with open('Data\PauliY', 'wb') as PY:
    pickle.dump(PauliY_propagators, PY)

    
with open('Data\PauliZ', 'wb') as PZ:
    pickle.dump(PauliZ_propagators, PZ)

    
with open('Data\Idle', 'wb') as ID:
    pickle.dump(unit_propagators, ID)

    
with open('Data\Time', 'wb') as ID:
    pickle.dump(t, ID)

In [6]:
# CZ Calculations
# Required CZs are CZ01, CZ12 and CZ13



# CZ01
SM1= tensor(qeye(2),  sigmam())
SM2= tensor(sigmam(), qeye(2))

SZ1= tensor(qeye(2),  sigmaz())
SZ2= tensor(sigmaz(), qeye(2))
              
c_ops=[]
gamma= dissipation[0]
Gamma= dephasing[0]
c_ops.append((sqrt(1/gamma)*SM1))
c_ops.append((sqrt(1/Gamma)*SZ1))

gamma= dissipation[1]
Gamma= dephasing[1]
c_ops.append((sqrt(1/gamma)*SM2))
c_ops.append((sqrt(1/Gamma)*SZ2))

T1= tensor(qeye(2),  qeye(2))
T2= tensor(qeye(2),  sigmaz())
T3= tensor(sigmaz(), qeye(2))
T4= tensor(sigmaz(), sigmaz())

H= (OmegaCZ/4.0)*(T1 - T2 - T3 + T4)
CZ_propagators= mesolve(H, qeye(H.dims), t, c_ops).states
CZ01= CZ_propagators[-1]




# CZ12
SM1= tensor(qeye(2),  sigmam())
SM2= tensor(sigmam(), qeye(2))

SZ1= tensor(qeye(2),  sigmaz())
SZ2= tensor(sigmaz(), qeye(2))
              
c_ops=[]
gamma= dissipation[1]
Gamma= dephasing[1]
c_ops.append((sqrt(1/gamma)*SM1))
c_ops.append((sqrt(1/Gamma)*SZ1))

gamma= dissipation[2]
Gamma= dephasing[2]
c_ops.append((sqrt(1/gamma)*SM2))
c_ops.append((sqrt(1/Gamma)*SZ2))

T1= tensor(qeye(2),  qeye(2))
T2= tensor(qeye(2),  sigmaz())
T3= tensor(sigmaz(), qeye(2))
T4= tensor(sigmaz(), sigmaz())

H= (OmegaCZ/4.0)*(T1 - T2 - T3 + T4)
CZ_propagators= mesolve(H, qeye(H.dims), t, c_ops).states
CZ12= CZ_propagators[-1]



# CZ13
SM1= tensor(qeye(2),  qeye(2), sigmam())
SM2= tensor(sigmam(), qeye(2), qeye(2))

SZ1= tensor(qeye(2),  qeye(2), sigmaz())
SZ2= tensor(sigmaz(), qeye(2), qeye(2))
              
c_ops=[]
gamma= dissipation[3]
Gamma= dephasing[3]
c_ops.append((sqrt(1/gamma)*SM1))
c_ops.append((sqrt(1/Gamma)*SZ1))

gamma= dissipation[1]
Gamma= dephasing[1]
c_ops.append((sqrt(1/gamma)*SM2))
c_ops.append((sqrt(1/Gamma)*SZ2))

T1= tensor(qeye(2),  qeye(2), qeye(2))
T2= tensor(qeye(2),  qeye(2), sigmaz())
T3= tensor(sigmaz(), qeye(2), qeye(2))
T4= tensor(sigmaz(), qeye(2), sigmaz())

H= (OmegaCZ/4.0)*(T1 - T2 - T3 + T4)
CZ_propagators= mesolve(H, qeye(H.dims), t, c_ops).states
CZ13= CZ_propagators[-1]



with open('Data\CZ01', 'wb') as ID:
    pickle.dump(CZ01, ID)
          
          
with open('Data\CZ12', 'wb') as ID:
    pickle.dump(CZ12, ID)
          
        
with open('Data\CZ13', 'wb') as ID:
    pickle.dump(CZ13, ID)